In [ ]:
########################################
# 1. SETUP & CONFIGURATION
########################################

# --- Hugging Face Login ---
from google.colab import userdata
from huggingface_hub import notebook_login

hf_token = userdata.get('HF_TOKEN')
if not hf_token:
    raise ValueError("HF_TOKEN not found in Colab Secrets. Please complete the prerequisite steps.")
notebook_login(new_session=hf_token)

!pip install vllm==0.7.3
!pip install -U transformers==4.53.2
!pip install -U peft
!pip install -U trl==0.20.0
!pip install -U accelerate
!pip install -U datasets
!pip install -U bitsandbytes
!pip install -q sympy
!pip install torch==2.6.0 torchvision --index-url https://download.pytorch.org/whl/test/cu124

# Install Flash Attention 2
!pip install flash-attn==2.7.4.post1 \
  --extra-index-url https://download.pytorch.org/whl/cu124 \
  --no-build-isolation


  Using cached torch-2.5.1-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached torchvision-0.20.1-cp311-cp311-manylinux1_x86_64.whl.metadata (6.1 kB)
  Using cached triton-3.1.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.3 kB)
Using cached torch-2.5.1-cp311-cp311-manylinux1_x86_64.whl (906.5 MB)
Using cached torchvision-0.20.1-cp311-cp311-manylinux1_x86_64.whl (7.2 MB)
Using cached triton-3.1.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (209.5 MB)
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: torch
    Found existing installation: torch 2.6.0+cu124
    Uninstalling torch-2.6.0+cu124:
      Successfully uninstalled torch-2.6.0+cu124
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.21.0+cu124
    Uninstalling torchvision-0.21.0+cu124:
      Successfully uninstalled torc

In [ ]:
# --- Experiment Configuration ---
from pathlib import Path
import json
import datetime
import torch
import random
import numpy as np

CONFIG = {
    # Core experiment parameters
    "experiment_type": "generative",
    "classification_type": "ternary",
    "dataset_strategy": "4N",
    "include_explanation": True,
    "include_eln": True,
    "solution_format": "nl",
    "model_name": "microsoft/phi-4-mini-instruct",

    # Prompting configuration
    "include_examples": True,
    "num_examples": 1,
    "example_strategy": "balanced",

    # Training parameters
    "learning_rate": 5e-5,
    "num_epochs": 3,
    "batch_size": 8,
    "inference_batch_size": 8, # Adjusted for few-shot prompt length
    "max_length": 2048, # Increased to accommodate few-shot examples
    "gradient_accumulation_steps": 4,

    # LoRa params
    "lora_rank": 16,
    "lora_alpha": 32,
    "lora_dropout": 0.1,

    # Paths and tokens
    "base_dataset_dir": "/content",
    "output_base_dir": "/content",

    # Experiment tracking
    "save_to_hf": True,
    "save_locally": True,
}

# --- Generate Unique Experiment ID ---
experiment_components = [
    CONFIG["experiment_type"][:4],
    CONFIG["classification_type"][:3],
    CONFIG["dataset_strategy"],
    "exp" if CONFIG["include_explanation"] else "no_exp",
    "eln" if CONFIG["include_eln"] else "no_eln",
    CONFIG["solution_format"],
    "phi4" if "phi" in CONFIG["model_name"].lower() else "qwen"
]
experiment_id = "_".join([c for c in experiment_components if c]) + "_" + datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
CONFIG["experiment_id"] = experiment_id
print(f"Experiment ID: {experiment_id}")

# --- Setup Output Directories ---
def setup_output_directory():
    output_dir = Path(CONFIG["output_base_dir"]) / CONFIG["experiment_id"]
    # Create the directory structure from the screenshot
    (output_dir / "baseline").mkdir(parents=True, exist_ok=True)
    (output_dir / "training").mkdir(parents=True, exist_ok=True)
    (output_dir / "final").mkdir(parents=True, exist_ok=True)

    config_path = output_dir / "config.json"
    with open(config_path, 'w') as f:
        json.dump(CONFIG, f, indent=2, default=str)
    print(f"Output directory created: {output_dir}")
    return output_dir

CONFIG["output_dir"] = setup_output_directory()

# --- Set Random Seeds for Reproducibility ---
def set_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

set_seeds(42)
print("✅ Setup complete. Dependencies installed and seeds set.")

Experiment ID: gene_ter_4N_exp_eln_nl_phi4_20250806_001250
Output directory created: /content/gene_ter_4N_exp_eln_nl_phi4_20250806_001250
✅ Setup complete. Dependencies installed and seeds set.


In [ ]:
SYSTEM_PROMPT = \
"""[ROLE]
You are a mathematics tutor.

[TASK]
You are given a math word problem and a solution written by a student.
Analyze the solution carefully, line-by-line, and classify it into one of the following categories:
- Correct: All logic is correct, and all calculations are correct
- Conceptual Error: There is an error in reasoning or logic somewhere in the solution
- Computational Error: All logic and reasoning is correct, but the result of some calculation is incorrect

[RESPONSE FORMAT]
Your response must contain exactly two parts, placed one after another with no additional text:

1. A thinking trace where you examine the solution line-by-line, ending with either "I found no errors!" or "Aha! I see the error."
This part must begin with "<think>" and end with "</think>".

2. A valid JSON object that follows this exact schema:
```json
{
  "verdict": "must be one of 'correct', 'conceptual_error', or 'computational_error'",
  "erroneous_line": "the exact, verbatim text of the VERY FIRST incorrect line, or null if the verdict is 'correct'",
  "explanation": "a brief, one-sentence explanation of the error"
}
```

Do NOT add any text before, after, or between these two parts.
"""

In [ ]:
########################################
# 2. CORE FUNCTION DEFINITIONS
########################################

import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training

def load_base_dataset():
    """Loads the base dataset from the specified CSV file."""
    base_dir = Path(CONFIG["base_dataset_dir"])
    dataset_file = base_dir / f"error_detection_dataset_with_traces.csv"
    # dataset_file = "../data/aug-5-dataset/error_detection_dataset_with_traces.csv"
    data = pd.read_csv(dataset_file)
    print(f"Loaded base {CONFIG['dataset_strategy']} dataset with {len(data)} samples")
    return data

def load_model():
    """Loads the quantized model."""
    model_name = CONFIG["model_name"]
    print(f"Loading model: {model_name}")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True
    )
    lora_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        r=CONFIG["lora_rank"],
        lora_alpha=CONFIG["lora_alpha"],
        lora_dropout=CONFIG["lora_dropout"],
        target_modules="all-linear",
        bias="none"
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
        attn_implementation="flash_attention_2"
    )
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, lora_config)

    model.generation_config = GenerationConfig(
        max_new_tokens=300,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        do_sample=False
    )

    return model

def load_tokenizer():
    """Loads the tokenizer."""
    model_name = CONFIG["model_name"]
    print(f"Loading tokenizer: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"
    return tokenizer

In [ ]:
##################################################
# 3. DATA PREPARATION (WITH FEW-SHOT EXAMPLES)
##################################################

from datasets import Dataset
import math

class ExampleManager:
    def __init__(self, base_dataset):
        self.samples = base_dataset.to_dict('records')
        self.config = CONFIG
        self._prepare_examples_by_problem()

    def _prepare_examples_by_problem(self):
        self.problems_by_type = {"correct": {}, "conceptual_error": {}, "computational_error": {}}
        for sample in self.samples:
            problem_index = sample["index"]
            error_type = sample["error_type"]
            if problem_index not in self.problems_by_type[error_type]:
                self.problems_by_type[error_type][problem_index] = []
            self.problems_by_type[error_type][problem_index].append(sample)

    def get_examples(self):
        """Returns examples based on dataset strategy"""
        if not self.config["include_examples"]:
            return []
        num_examples = self.config["num_examples"]
        dataset_strategy = self.config["dataset_strategy"]
        examples = []

        import random
        if dataset_strategy == "3N":
            # Choose num_examples distinct problem indices that have all 3 versions
            available_problems = set(self.problems_by_type["correct"].keys()) & \
                               set(self.problems_by_type["conceptual_error"].keys()) & \
                               set(self.problems_by_type["computational_error"].keys())
            # Sample problem indices
            selected_problems = random.sample(list(available_problems), num_examples)
            for problem_index in selected_problems:
                # Add all 3 versions: correct, conceptual_error, computational_error
                examples.append(self.problems_by_type["correct"][problem_index][0])
                examples.append(self.problems_by_type["conceptual_error"][problem_index][0])
                examples.append(self.problems_by_type["computational_error"][problem_index][0])

            return examples

        elif dataset_strategy == "4N":
            import math
            # Get problems that have conceptual errors (with correct versions)
            conceptual_problems = list(
                set(self.problems_by_type["correct"].keys()) &
                set(self.problems_by_type["conceptual_error"].keys())
            )
            # Get problems that have computational errors (with correct versions)
            computational_problems = list(
                set(self.problems_by_type["correct"].keys()) &
                set(self.problems_by_type["computational_error"].keys())
            )
            # Calculate splits: floor(n/2) conceptual, ceil(n/2) computational
            n_conceptual = num_examples // 2  # This is floor(n/2)
            n_computational = math.ceil(num_examples / 2)

            # Sample conceptual problems
            if conceptual_problems and n_conceptual > 0:
                selected_conceptual = random.sample(conceptual_problems,n_conceptual)
                for problem_index in selected_conceptual:
                    # Add correct + conceptual_error pair
                    examples.append(self.problems_by_type["correct"][problem_index][0])
                    examples.append(self.problems_by_type["conceptual_error"][problem_index][0])

            # Sample computational problems
            if computational_problems and n_computational > 0:
                selected_computational = random.sample(computational_problems,n_computational)
                for problem_index in selected_computational:
                    # Add correct + computational_error pair
                    examples.append(self.problems_by_type["correct"][problem_index][0])
                    examples.append(self.problems_by_type["computational_error"][problem_index][0])

            return examples

        else:
            print(f"Warning: Unknown dataset strategy '{dataset_strategy}'")
            return []

def format_user_message(sample):
    """Formats a sample into a user message string."""
    question = sample.get('question', '') or ''
    solution = sample.get('correct_answer' if sample['error_type'] == 'correct' else 'wrong_answer', '').strip()
    return f"### Question:\n{question}\n\n### Answer:\n{solution}"

def format_expected_output(sample):
    """Creates the expected output string for a sample."""
    json_output_string = sample.get('json_output_string', '').strip()

    # Concatenate thinking trace with JSON output
    return thinking_trace + '\n' + "```json\n" + json_output_string + "\n```"

def create_sample_messages(sample, examples):
    """Creates the full message list for a sample, including few-shot examples."""
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    for example in examples:
        messages.append({"role": "user", "content": format_user_message(example)})
        messages.append({"role": "assistant", "content": format_expected_output(example)})
    messages.append({"role": "user", "content": format_user_message(sample)})
    return messages

def prepare_data_for_run(tokenizer, base_df):
    """Prepares data, generates examples, and returns datasets."""
    test_df = base_df.sample(frac=0.2, random_state=42)
    train_df = base_df.drop(test_df.index)
    print(f"Data split: {len(train_df)} training samples, {len(test_df)} test samples.")

    example_manager = ExampleManager(base_df)
    examples = example_manager.get_examples()
    print(f"Generated {len(examples)} few-shot examples for this run.")

    def create_training_text(sample):
        # Re-use the message creation logic, now including examples
        messages = create_sample_messages(sample, examples)
        # Add the final assistant response for the training label
        messages.append({"role": "assistant", "content": format_expected_output(sample)})
        return {"text": tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)}

    train_dataset = Dataset.from_pandas(train_df)
    train_dataset_formatted = train_dataset.map(create_training_text, load_from_cache_file=False)
    train_dataset_tokenized = train_dataset_formatted.map(
        lambda x: tokenizer(
            x["text"],
            truncation=True,
            max_length=CONFIG["max_length"],
            padding=False),
        remove_columns=['text'] + list(train_df.columns)
    )
    return train_dataset_tokenized, test_df, examples

print("✅ Data preparation functions (with ExampleManager) loaded.")

✅ Data preparation functions (with ExampleManager) loaded.


In [ ]:
########################################
# 4. INFERENCE & EVALUATION FUNCTIONS
########################################

from vllm import LLM, SamplingParams
import math
import re
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def run_vllm_inference_on_dataframe(model_name_or_path, tokenizer, df, examples):
    """Runs highly optimized inference using the vLLM engine."""
    print(f"\n--- Initializing vLLM engine for model: {model_name_or_path} ---")

    llm = LLM(
        model=model_name_or_path,
        trust_remote_code=True,
        max_model_len=2048,
        gpu_memory_utilization=0.90
    )

    sampling_params = SamplingParams(temperature=0, max_tokens=300)

    print("--- Preparing prompts for vLLM ---")
    all_prompts = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Formatting Prompts"):
        messages = create_sample_messages(row.to_dict(), examples)
        prompt_string = tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
        all_prompts.append(prompt_string)

    print(f"\n--- Running vLLM inference on {len(all_prompts)} prompts ---")
    vllm_outputs = llm.generate(all_prompts, sampling_params)

    predictions = [output.outputs[0].text for output in vllm_outputs]
    print("--- vLLM Inference complete ---")
    return predictions

def extract_thinking_from_response(response):
    """Extracts the thinking trace from a model's text response."""
    match = re.search(r'<think>(.*?)</think>', response, re.DOTALL)
    return match.group(1).strip() if match else ""

def extract_json_from_response(response):
    """Extracts a JSON object from a model's text response."""
    match = re.search(r'\{.*\}', response, re.DOTALL)
    return json.loads(match.group(0)) if match else {}

def create_detailed_results(test_df, predictions):
    """Merges model predictions with original test data."""
    results = []
    for i, pred_text in enumerate(predictions):
        original_sample = test_df.iloc[i].to_dict()
        parsed_json = extract_json_from_response(pred_text)
        predicted_thinking = extract_thinking_from_response(pred_text)

        # Get expected values
        expected_thinking = original_sample.get('thinking_trace', '').replace('<think>', '').replace('</think>', '').strip()
        try:
            expected_json = json.loads(original_sample.get('json_output_string', '{}'))
        except (json.JSONDecodeError, TypeError):
            expected_json = {}

        result_row = {
            'problem_index': original_sample.get('index'),
            'expected_verdict': original_sample.get('error_type'),
            'predicted_verdict': parsed_json.get('verdict'),
            'verdict_correct': original_sample.get('error_type') == parsed_json.get('verdict'),
            'expected_erroneous_line': expected_json.get('erroneous_line'),
            'predicted_erroneous_line': parsed_json.get('erroneous_line'),
            'expected_thinking': expected_thinking,
            'predicted_thinking': predicted_thinking,
            'full_prediction_text': pred_text.strip(),
        }
        results.append(result_row)
    return pd.DataFrame(results)

def normalize_text(text):
    """Normalizes text by removing all whitespace and converting to lowercase."""
    return re.sub(r'\s+', '', str(text).lower().strip()) if text else ""

def compute_metrics_from_results(results_df):
    """Calculates verdict and erroneous line accuracy from the results DataFrame."""
    y_true_verdict = results_df['expected_verdict']
    y_pred_verdict = results_df['predicted_verdict'].fillna('parse_failure')
    accuracy = accuracy_score(y_true_verdict, y_pred_verdict)
    labels = list(y_true_verdict.unique()) + ['parse_failure']
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_true_verdict,
        y_pred_verdict,
        average='macro',
        zero_division=0,
        labels=labels)

    error_rows = results_df[results_df['expected_verdict'].isin(['conceptual_error', 'computational_error'])].copy()
    eln_accuracy = accuracy_score(
        error_rows['expected_erroneous_line'].apply(normalize_text),
        error_rows['predicted_erroneous_line'].apply(normalize_text)
        ) if not error_rows.empty else 1.0

    # Count different types of parse failures
    json_parse_failures = int(results_df['predicted_verdict'].isnull().sum())

    # Count thinking parse failures (empty thinking traces)
    thinking_parse_failures = 0
    if 'predicted_thinking' in results_df.columns:
        thinking_parse_failures = int(results_df['predicted_thinking'].isnull().sum()) + \
                                 int((results_df['predicted_thinking'] == '').sum())

    return {
        "overall_accuracy": accuracy,
        "precision_macro": precision,
        "recall_macro": recall,
        "f1_macro": f1,
        "eln_accuracy_on_errors": eln_accuracy,
        "total_samples": len(results_df),
        "error_samples_for_eln": len(error_rows),
        "json_parse_failures": json_parse_failures,
        "thinking_parse_failures": thinking_parse_failures
    }

print("✅ Inference and evaluation functions loaded.")

✅ Inference and evaluation functions loaded.


In [ ]:
########################################
# 5. FINE-TUNING LOOP
########################################

from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

def run_fine_tuning(model, tokenizer, train_dataset):
    """
    Runs a focused fine-tuning process without intermediate evaluation.
    """
    output_dir = CONFIG["output_dir"]

    training_args = TrainingArguments(
        output_dir=str(output_dir / "training_run"),
        num_train_epochs=CONFIG["num_epochs"],
        per_device_train_batch_size=CONFIG["batch_size"],
        gradient_accumulation_steps=CONFIG["gradient_accumulation_steps"],
        optim="paged_adamw_8bit",
        learning_rate=CONFIG["learning_rate"],
        lr_scheduler_type="cosine",
        warmup_ratio=0.1,
        bf16=True,
        logging_steps=10,
        report_to="none",
        eval_strategy="no",
        save_strategy="epoch", # Save adapter at the end of each epoch
        save_total_limit=1, # Only keep the best/last checkpoint
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        processing_class=tokenizer,
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
    )

    print(f"\n--- Starting fine-tuning for {CONFIG['num_epochs']} epochs ---")
    trainer.train()

    # Save the final adapter to a dedicated directory
    final_adapter_dir = output_dir / "final_adapter"
    trainer.save_model(str(final_adapter_dir))
    print(f"\n✅✅✅ Fine-tuning finished! Final adapter saved to {final_adapter_dir} ✅✅✅")

print("✅ Fine-tuning function loaded.")

✅ Fine-tuning function loaded.


In [ ]:
##########
# NEW: MATH EVALUATION TOOL
##########

import sympy
import re

def evaluate_equation(equation_str: str) -> tuple[bool, str]:
    """
    Safely evaluates a mathematical equation string.

    Args:
        equation_str: A string like "5 * 2 = 12" or "10 + 5 = 15".

    Returns:
        A tuple containing:
        - bool: True if the equation is mathematically correct, False otherwise.
        - str: The actual computed result of the left-hand side.
    """
    try:
        # 1. Clean and split the equation
        # Handles potential floating point issues and extra spaces
        if '=' not in equation_str:
            return False, "Malformed equation"
        
        lhs_str, rhs_str = equation_str.split('=', 1)
        
        # Remove any text that is not part of the math expression
        lhs_str = re.sub(r'[^\d\s\.\+\-\*\/(\)]', '', lhs_str).strip()
        rhs_str = re.sub(r'[^\d\s\.\+\-\*\/(\)]', '', rhs_str).strip()

        # 2. Use sympy to safely parse the expressions
        lhs_expr = sympy.parse_expr(lhs_str, evaluate=True)
        rhs_expr = sympy.parse_expr(rhs_str, evaluate=True)

        # 3. Compare the results
        # Use a small tolerance for floating point comparisons
        is_correct = abs(lhs_expr - rhs_expr) < 1e-9
        
        return is_correct, str(lhs_expr)
    except (sympy.SympifyError, IndexError, TypeError, SyntaxError):
        # If parsing fails, it's not a valid or evaluatable equation
        return False, "Invalid expression"

print("✅ Math evaluation tool loaded.")
# Test cases
print(f"'10 + 5 = 15' -> {evaluate_equation('10 + 5 = 15')}")
print(f"'5 * 2 = 12'   -> {evaluate_equation('5 * 2 = 12')}")
print(f"'10/2 = 5.0' -> {evaluate_equation('10/2 = 5.0')}")

In [ ]:
# ########################################
# # PIPELINE STEP 1: INITIALIZATION
# ########################################

tokenizer = load_tokenizer()
base_df = load_base_dataset()
train_dataset, test_df, examples = prepare_data_for_run(tokenizer, base_df)

Loading tokenizer: microsoft/phi-4-mini-instruct
Loaded base 4N dataset with 6067 samples
Data split: 4854 training samples, 1213 test samples.
Generated 2 few-shot examples for this run.


Map:   0%|          | 0/4854 [00:00<?, ? examples/s]

Map:   0%|          | 0/4854 [00:00<?, ? examples/s]

In [ ]:
##########
# VIEW A SAMPLE PROMPT
##########

print("--- Displaying a complete sample prompt for verification ---\n")

# Make sure the necessary variables are available from previous cells
if 'test_df' not in locals() or 'examples' not in locals() or 'tokenizer' not in locals():
    print("Please run the 'PIPELINE STEP 1: INITIALIZATION' cell first to load data.")
else:
    # Select the first sample from the test set to use for this example
    sample_to_view = test_df.iloc[0].to_dict()
    print(f"--- Using Sample from Problem Index: {sample_to_view.get('index')} ---")
    print(f"--- Expected Verdict: {sample_to_view.get('error_type')} ---\n")

    # Use the existing helper function to construct the full message list,
    # including the system prompt and the few-shot examples.
    sample_messages = create_sample_messages(sample_to_view, examples)

    # Use the tokenizer's chat template to render the final string.
    # - tokenize=False returns the formatted string.
    # - add_generation_prompt=True adds the final special tokens that signal
    #   to the model that it is its turn to generate a response.
    final_prompt_string = tokenizer.apply_chat_template(
        sample_messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Print the final, complete prompt
    print("--- FULL PROMPT AS SEEN BY THE MODEL ---")
    print(final_prompt_string)

--- Displaying a complete sample prompt for verification ---

--- Using Sample from Problem Index: 7385 ---
--- Expected Verdict: computational_error ---

--- FULL PROMPT AS SEEN BY THE MODEL ---
<|system|>[ROLE]
You are a mathematics tutor.

[TASK]
You are given a math word problem and a solution written by a student.
Analyze the solution carefully, line-by-line, and classify it into one of the following categories:
- Correct: All logic is correct, and all calculations are correct
- Conceptual Error: There is an error in reasoning or logic somewhere in the solution
- Computational Error: All logic and reasoning is correct, but the result of some calculation is incorrect

[RESPONSE FORMAT]
Your response must contain exactly two parts, placed one after another with no additional text:

1. A thinking trace where you examine the solution line-by-line, ending with either "I found no errors!" or "Aha! I see the error."
This part must begin with "<think>" and end with "</think>".

2. A valid

In [ ]:
# ##########
# # TOKEN COUNT STATISTICS
# ##########

# from tqdm.auto import tqdm

# def compute_token_statistics(tokenizer, base_df, examples):
#     """
#     Compute token count statistics for inputs and expected outputs to help set max_length.
#     """
#     print("🔢 Computing token count statistics...")

#     input_token_counts = []
#     output_token_counts = []
#     total_token_counts = []

#     # Sample a subset for faster computation if dataset is large
#     sample_size = len(base_df)
#     sample_df = base_df.sample(n=sample_size, random_state=42)

#     print(f"Analyzing {sample_size} samples...")

#     for _, row in tqdm(sample_df.iterrows(), total=len(sample_df), desc="Computing token counts"):
#         sample_dict = row.to_dict()

#         # Create input messages (system + examples + user query)
#         input_messages = create_sample_messages(sample_dict, examples)
#         input_text = tokenizer.apply_chat_template(
#             input_messages,
#             tokenize=False,
#             add_generation_prompt=True
#         )

#         # Get expected output
#         expected_output = format_expected_output(sample_dict)

#         # Tokenize and count
#         input_tokens = tokenizer.encode(input_text, add_special_tokens=False)
#         output_tokens = tokenizer.encode(expected_output, add_special_tokens=False)

#         input_token_counts.append(len(input_tokens))
#         output_token_counts.append(len(output_tokens))
#         total_token_counts.append(len(input_tokens) + len(output_tokens))

#     # Convert to numpy arrays for statistics
#     import numpy as np
#     input_counts = np.array(input_token_counts)
#     output_counts = np.array(output_token_counts)
#     total_counts = np.array(total_token_counts)

#     print("\n" + "="*60)
#     print("📊 TOKEN COUNT STATISTICS")
#     print("="*60)

#     print(f"\n🔤 INPUT TOKEN COUNTS (System + Examples + User Query):")
#     print(f"  Mean: {input_counts.mean():.1f}")
#     print(f"  Median: {np.median(input_counts):.1f}")
#     print(f"  Min: {input_counts.min()}")
#     print(f"  Max: {input_counts.max()}")
#     print(f"  95th percentile: {np.percentile(input_counts, 95):.1f}")
#     print(f"  99th percentile: {np.percentile(input_counts, 99):.1f}")

#     print(f"\n📝 OUTPUT TOKEN COUNTS (Expected Response):")
#     print(f"  Mean: {output_counts.mean():.1f}")
#     print(f"  Median: {np.median(output_counts):.1f}")
#     print(f"  Min: {output_counts.min()}")
#     print(f"  Max: {output_counts.max()}")
#     print(f"  95th percentile: {np.percentile(output_counts, 95):.1f}")
#     print(f"  99th percentile: {np.percentile(output_counts, 99):.1f}")

#     print(f"\n🔗 TOTAL TOKEN COUNTS (Input + Output):")
#     print(f"  Mean: {total_counts.mean():.1f}")
#     print(f"  Median: {np.median(total_counts):.1f}")
#     print(f"  Min: {total_counts.min()}")
#     print(f"  Max: {total_counts.max()}")
#     print(f"  95th percentile: {np.percentile(total_counts, 95):.1f}")
#     print(f"  99th percentile: {np.percentile(total_counts, 99):.1f}")

#     print(f"\n📈 RECOMMENDED MAX_LENGTH SETTINGS:")
#     print(f"  Conservative (99th percentile): {int(np.percentile(total_counts, 99))}")
#     print(f"  Balanced (95th percentile): {int(np.percentile(total_counts, 95))}")
#     print(f"  Aggressive (90th percentile): {int(np.percentile(total_counts, 90))}")

#     print(f"\n🎯 TRUNCATION IMPACT ANALYSIS:")
#     for percentile in [90, 95, 99]:
#         threshold = np.percentile(total_counts, percentile)
#         truncated = (total_counts > threshold).sum()
#         percentage = (truncated / len(total_counts)) * 100
#         print(f"  Max length {int(threshold)}: {truncated}/{len(total_counts)} samples truncated ({percentage:.1f}%)")

#     print("\n" + "="*60)

#     # Return the statistics for further analysis
#     return {
#         'input_stats': {
#             'mean': float(input_counts.mean()),
#             'median': float(np.median(input_counts)),
#             'min': int(input_counts.min()),
#             'max': int(input_counts.max()),
#             'p95': float(np.percentile(input_counts, 95)),
#             'p99': float(np.percentile(input_counts, 99))
#         },
#         'output_stats': {
#             'mean': float(output_counts.mean()),
#             'median': float(np.median(output_counts)),
#             'min': int(output_counts.min()),
#             'max': int(output_counts.max()),
#             'p95': float(np.percentile(output_counts, 95)),
#             'p99': float(np.percentile(output_counts, 99))
#         },
#         'total_stats': {
#             'mean': float(total_counts.mean()),
#             'median': float(np.median(total_counts)),
#             'min': int(total_counts.min()),
#             'max': int(total_counts.max()),
#             'p95': float(np.percentile(total_counts, 95)),
#             'p99': float(np.percentile(total_counts, 99))
#         }
#     }

# # Run the analysis
# token_stats = compute_token_statistics(tokenizer, base_df, examples)

# # Optional: Show distribution histogram
# def plot_token_distribution(tokenizer, base_df, examples):
#     """Plot token count distribution"""
#     import matplotlib.pyplot as plt

#     # Compute token counts for a sample
#     sample_size = min(500, len(base_df))
#     sample_df = base_df.sample(n=sample_size, random_state=42)

#     total_counts = []
#     for _, row in sample_df.iterrows():
#         sample_dict = row.to_dict()
#         input_messages = create_sample_messages(sample_dict, examples)
#         input_text = tokenizer.apply_chat_template(input_messages, tokenize=False, add_generation_prompt=True)
#         expected_output = format_expected_output(sample_dict)

#         input_tokens = len(tokenizer.encode(input_text, add_special_tokens=False))
#         output_tokens = len(tokenizer.encode(expected_output, add_special_tokens=False))
#         total_counts.append(input_tokens + output_tokens)

#     plt.figure(figsize=(10, 6))
#     plt.hist(total_counts, bins=30, alpha=0.7, edgecolor='black')
#     plt.axvline(np.percentile(total_counts, 95), color='red', linestyle='--', label='95th percentile')
#     plt.axvline(np.percentile(total_counts, 99), color='orange', linestyle='--', label='99th percentile')
#     plt.xlabel('Total Token Count')
#     plt.ylabel('Frequency')
#     plt.title('Distribution of Total Token Counts')
#     plt.legend()
#     plt.grid(True, alpha=0.3)
#     plt.show()

# # Uncomment to see the distribution plot
# # plot_token_distribution(tokenizer, base_df, examples)

In [ ]:
# ##########
# # PIPELINE STEP 2: BASELINE EVALUATION (with vLLM)
# ##########

# import gc

# baseline_predictions = run_vllm_inference_on_dataframe(CONFIG["model_name"], tokenizer, test_df, examples)
# baseline_results_df = create_detailed_results(test_df, baseline_predictions)
# baseline_metrics = compute_metrics_from_results(baseline_results_df)

# print("\n--- Baseline Metrics ---")
# print(json.dumps(baseline_metrics, indent=2))

# # Save results
# baseline_dir = CONFIG["output_dir"] / "baseline"
# baseline_results_df.to_csv(baseline_dir / "results_baseline.csv", index=False)
# with open(baseline_dir / "metrics_baseline.json", 'w') as f:
#     json.dump(baseline_metrics, f, indent=2)
# print(f"Baseline results saved to {baseline_dir}")

# # --- CRUCIAL: Release vLLM resources ---
# # We must delete the llm object created inside the function, which is now out of scope.
# # Triggering garbage collection and emptying the cache ensures the GPU is free.
# print("\nReleasing vLLM engine from GPU memory...")
# gc.collect()
# torch.cuda.empty_cache()
# print("✅ GPU memory cleared.")

In [ ]:
##########
# PIPELINE STEP 3: RUN FINE-TUNING
##########

# Load a fresh model specifically for training
training_model = load_model()

run_fine_tuning(training_model, tokenizer, train_dataset)

# --- CRUCIAL: Release training resources ---
print("\nReleasing training model from GPU memory...")
del training_model
gc.collect()
torch.cuda.empty_cache()
print("✅ GPU memory cleared.")

Loading model: microsoft/phi-4-mini-instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.



--- Starting fine-tuning for 3 epochs ---


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.


Step,Training Loss
10,1.522400
20,1.184100
30,0.787900
40,0.441200
50,0.281600
60,0.219000
70,0.187900
80,0.174700
90,0.161400
100,0.155500


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)



✅✅✅ Fine-tuning finished! Final adapter saved to /content/gene_ter_4N_exp_eln_nl_phi4_20250806_001250/final_adapter ✅✅✅

Releasing training model from GPU memory...


NameError: name 'gc' is not defined

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
##########
# PIPELINE STEP 4: FINAL EVALUATION ON BEST MODEL (with vLLM)
##########
from peft import PeftModel

# --- Merge the adapter with the base model ---
print("\nMerging fine-tuned adapter with the base model...")
adapter_dir = CONFIG["output_dir"] / "final_adapter"
merged_model_dir = CONFIG["output_dir"] / "final_merged_model"

# Load base model on CPU to avoid using GPU memory during merge
base_model = AutoModelForCausalLM.from_pretrained(
    CONFIG["model_name"],
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
)
model_with_adapter = PeftModel.from_pretrained(base_model, str(adapter_dir))
merged_model = model_with_adapter.merge_and_unload()
merged_model.save_pretrained(str(merged_model_dir))
tokenizer.save_pretrained(str(merged_model_dir))
print(f"Merged model saved to {merged_model_dir}")

# --- Clear memory before loading vLLM ---
del base_model, model_with_adapter, merged_model
gc.collect()
torch.cuda.empty_cache()

# --- Run Final Inference on Merged Model ---
final_predictions = run_vllm_inference_on_dataframe(str(merged_model_dir), tokenizer, test_df, examples)
final_results_df = create_detailed_results(test_df, final_predictions)
final_metrics = compute_metrics_from_results(final_results_df)

print("\n--- Final Metrics (Best Model) ---")
print(json.dumps(final_metrics, indent=2))

final_dir = CONFIG["output_dir"] / "final"
final_results_df.to_csv(final_dir / "results_final_best.csv", index=False)
with open(final_dir / "metrics_final_best.json", 'w') as f:
    json.dump(final_metrics, f, indent=2)
print(f"Final results saved to {final_dir}")


Merging fine-tuned adapter with the base model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Merged model saved to /content/gene_ter_4N_exp_eln_nl_phi4_20250806_001250/final_merged_model

--- Initializing vLLM engine for model: /content/gene_ter_4N_exp_eln_nl_phi4_20250806_001250/final_merged_model ---
INFO 08-06 01:39:04 __init__.py:207] Automatically detected platform cuda.
INFO 08-06 01:39:04 config.py:208] Replacing legacy 'type' key with 'rope_type'
INFO 08-06 01:39:16 config.py:549] This model supports multiple tasks: {'classify', 'reward', 'score', 'embed', 'generate'}. Defaulting to 'generate'.
INFO 08-06 01:39:16 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='/content/gene_ter_4N_exp_eln_nl_phi4_20250806_001250/final_merged_model', speculative_config=None, tokenizer='/content/gene_ter_4N_exp_eln_nl_phi4_20250806_001250/final_merged_model', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=2048, download_dir=None, loa

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 08-06 01:39:21 model_runner.py:1115] Loading model weights took 7.1694 GB
INFO 08-06 01:39:22 worker.py:267] Memory profiling takes 0.75 seconds
INFO 08-06 01:39:22 worker.py:267] the current vLLM instance can use total_gpu_memory (39.56GiB) x gpu_memory_utilization (0.90) = 35.60GiB
INFO 08-06 01:39:22 worker.py:267] model weights take 7.17GiB; non_torch_memory takes 0.01GiB; PyTorch activation peak memory takes 1.83GiB; the rest of the memory reserved for KV Cache is 26.60GiB.
INFO 08-06 01:39:23 executor_base.py:111] # cuda blocks: 13618, # CPU blocks: 2048
INFO 08-06 01:39:23 executor_base.py:116] Maximum concurrency for 2048 tokens per request: 106.39x
INFO 08-06 01:39:25 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_u

...


--- Running vLLM inference on 1213 prompts ---


Processed prompts:   0%|          | 0/1213 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

WARNING 08-06 01:40:11 scheduler.py:1754] Sequence group 242 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1


Processed prompts: 100%|██████████| 1213/1213 [01:41<00:00, 11.92it/s, est. speed input: 10468.96 toks/s, output: 1399.31 toks/s]


--- vLLM Inference complete ---

--- Final Metrics (Best Model) ---
{
  "overall_accuracy": 0.2943116240725474,
  "precision_macro": 0.07940391459074733,
  "recall_macro": 0.24586776859504134,
  "f1_macro": 0.12004034969737727,
  "eln_accuracy_on_errors": 0.04941176470588235,
  "total_samples": 1213,
  "error_samples_for_eln": 850,
  "json_parse_failures": 0,
  "thinking_parse_failures": 0
}
Final results saved to /content/gene_ter_4N_exp_eln_nl_phi4_20250806_001250/final


In [ ]:
##########
# PIPELINE STEP 6: COMPRESS RESULTS FOR DOWNLOAD
##########

import zipfile
from pathlib import Path

print("Compressing experiment results...")

# Define the root directory of your experiment
root_dir = Path(CONFIG["output_dir"])
zip_path = root_dir / f"{CONFIG['experiment_id']}_results.zip"

# Define the folder to exclude
exclude_dir = root_dir / "final_merged_model"

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Walk through all the files and folders in the root directory
    for file_path in root_dir.rglob('*'):
        # Check if the current file_path is inside the excluded directory
        if exclude_dir in file_path.parents or file_path == exclude_dir:
            continue # Skip this file or folder

        # Check if the path is the zip file itself to avoid recursion
        if file_path == zip_path:
            continue

        # Write the file to the zip archive with a relative path
        arcname = file_path.relative_to(root_dir)
        zipf.write(file_path, arcname)

print(f"\n✅ Successfully created zip archive at: {zip_path}")
print("You can now download this file from the file browser on the left.")

In [ ]:
final_results_df.sort_values(by='problem_index', inplace=True)

In [ ]:
final_results_df.head()

,problem_index,expected_verdict,predicted_verdict,verdict_correct,expected_erroneous_line,predicted_erroneous_line,expected_thinking,predicted_thinking,full_prediction_text
0,7385,computational_error,correct,False,That means he sold it for 4050*1.1=$4545,None,"There are 5 lines in the solution, including t...","There are 6 lines in the solution, including t...","<think>\nThere are 6 lines in the solution, in..."
1,2668,computational_error,correct,False,"So, Hannah's family has 20 - 3 = 20 cows.",None,"There are 5 lines in the solution, including t...","There are 5 lines in the solution, including t...","<think>\nThere are 5 lines in the solution, in..."
2,858,conceptual_error,correct,False,This means he took in 300+125 = 425 calories f...,None,"There are 5 lines in the solution, including t...","There are 6 lines in the solution, including t...","<think>\nThere are 6 lines in the solution, in..."
3,3796,computational_error,correct,False,He has 70 - 9 = $16 left after buying the fris...,None,"There are 3 lines in the solution, including t...","There are 4 lines in the solution, including t...","<think>\nThere are 4 lines in the solution, in..."
4,2243,correct,correct,True,None,None,"There are 4 lines in the solution, including t...","There are 4 lines in the solution, including t...","<think>\nThere are 4 lines in the solution, in..."


In [ ]:
temp = final_results_df.head()
for index, row in temp.iterrows():
    print(row['problem_index'])
    print(row['expected_verdict'])
    print(row['predicted_verdict'])
    print(row['predicted_thinking'])
    print(row['expected_thinking'])

7385
computational_error
correct
There are 6 lines in the solution, including the final answer line. Let's examine the lines one-by-one.
Line 1: no errors
Line 2: no errors
Line 3: no errors
Line 4: no errors
Line 5: no errors
Final Answer: no errors
I found no errors! I will now prepare the output json.
There are 5 lines in the solution, including the final answer line. Let's examine the lines one-by-one.
Line 1: no errors
Line 2: no errors
Line 3: no errors
Line 4: Aha! I see the error. I will now prepare the output json.
2668
computational_error
correct
There are 5 lines in the solution, including the final answer line. Let's examine the lines one-by-one.
Line 1: no errors
Line 2: no errors
Line 3: no errors
Line 4: no errors
Final Answer: no errors
I found no errors! I will now prepare the output json.
There are 5 lines in the solution, including the final answer line. Let's examine the lines one-by-one.
Line 1: no errors
Line 2: Aha! I see the error. I will now prepare the output 

In [ ]:
##########
# CONFUSION MATRIX ANALYSIS
##########
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import pandas as pd

print("--- Generating Confusion Matrix for Final Model Verdicts ---")

# Ensure the final results DataFrame is available
if 'final_results_df' not in locals():
    print("Error: `final_results_df` not found. Please run the final evaluation cell first.")
else:
    # Define the order of labels for a consistent matrix layout
    labels = ['correct', 'conceptual_error', 'computational_error', 'parse_failure']

    # Extract the true and predicted values, filling any parse failures
    y_true = final_results_df['expected_verdict']
    y_pred = final_results_df['predicted_verdict'].fillna('parse_failure')

    # Compute the confusion matrix
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    # Create a DataFrame for better labeling with seaborn
    cm_df = pd.DataFrame(cm, index=labels, columns=labels)

    # Plot the heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues',
                annot_kws={"size": 14}) # Increase annotation font size

    plt.title('Verdict Confusion Matrix (Final Model)', fontsize=16)
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.show()

In [ ]:
# ##########
# # PIPELINE STEP 5: PUSH TO HUB
# ##########

# if CONFIG["save_to_hf"]:
#     print("\nPushing the final MERGED model to the Hugging Face Hub...")
#     # The merged model can be pushed directly using its save directory
#     hf_repo_id = CONFIG["experiment_id"]
#     merged_model_dir = str(CONFIG["output_dir"] / "final_merged_model")

#     # Use the API to upload the folder
#     from huggingface_hub import HfApi
#     api = HfApi()
#     api.upload_folder(
#         folder_path=merged_model_dir,
#         repo_id=hf_repo_id,
#         repo_type="model"
#     )
#     print(f"✅ Final merged model pushed to Hub repo: {hf_repo_id}")
# else:
#     print("\nSkipping Hugging Face Hub upload as per configuration.")

In [ ]:
##########
# 2. UNZIP YOUR SAVED RESULTS
##########
import zipfile
from pathlib import Path

# IMPORTANT: Replace this with the actual name of your uploaded zip file
zip_filename = "gene_ter_3N_exp_eln_nl_phi4_20250803_174501_results.zip" #<-- CHANGE THIS

zip_path = Path("/content/") / zip_filename

if not zip_path.exists():
    raise FileNotFoundError(f"'{zip_filename}' not found. Please upload the file first.")

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/")

# The unzipped folder name is the experiment ID
experiment_id = zip_filename.replace("_results.zip", "")
experiment_dir = Path("/content/") / experiment_id
print(f"✅ Successfully unzipped results to: {experiment_dir}")

##########
# 3. LOAD, MERGE, AND PUSH
##########
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# --- Configuration ---
# We can get these from your unzipped folder name and a config file
# or define them manually.
BASE_MODEL_NAME = "microsoft/phi-4-mini-instruct" # Or whatever you used
ADAPTER_DIR = experiment_dir / "final_best_model" # Or final_adapter
HF_REPO_ID = experiment_id # Use the experiment ID as the repo name

# --- Load Base Model and Adapter ---
print("Loading base model for merging...")
# Load on the CPU to be memory-efficient
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
)

print(f"Loading adapter from: {ADAPTER_DIR}")
# Apply the saved LoRA adapter to the base model
model_with_adapter = PeftModel.from_pretrained(base_model, str(ADAPTER_DIR))

# --- Merge the Weights ---
print("Merging adapter weights into the base model...")
merged_model = model_with_adapter.merge_and_unload()
print("Merge complete.")

# --- Load the Tokenizer ---
# The tokenizer was saved alongside the adapter
tokenizer = AutoTokenizer.from_pretrained(ADAPTER_DIR)

# --- Push to Hugging Face Hub ---
print(f"Pushing final merged model and tokenizer to Hub repo: {HF_REPO_ID}")
merged_model.push_to_hub(HF_REPO_ID)
tokenizer.push_to_hub(HF_REPO_ID)

print(f"\n✅✅✅ Successfully uploaded model to: https://huggingface.co/{HF_REPO_ID}")